In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
train = pd.read_csv('data/train.csv', index_col=0)
train.head()

,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
ID,,,,,,,,,,,,,,,,,,,,,
000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,2200000.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,2000000.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [3]:
X = train.drop('target', axis=1)
y = train.target

In [4]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=500, n_iter=10, random_state=42)
X_svd = svd.fit_transform(X)
X_svd.shape

(4459, 500)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_svd, y, test_size=0.2, random_state=42
)

In [6]:
def rmsle_metric(y_pred,y_test) : 
    assert len(y_test) == len(y_pred)
    return np.sqrt(np.mean((np.log(1+y_pred) - np.log(1+y_test))**2))

### Linear Regression

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

params = {
    'normalize': [True, False],
    'fit_intercept': [True, False]
}

lr = LinearRegression()
lr_gs = GridSearchCV(lr, params)
lr_gs.fit(X_train, y_train)

y_pred = lr_gs.predict(X_test)

print(rmsle_metric(y_pred, y_test))

lr_gs.best_estimator_

1.9352003602287482


/home/rafael/.virtualenvs/kaggle-santander/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

### SVM

In [7]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

params = {
    'C': [.5, 1.5],
    'epsilon': [0.05, 0.1, 0.3],
    'kernel': ['poly', 'rbf'],
}

svr = SVR()
svr_gs = GridSearchCV(svr, params, n_jobs=3)
svr_gs.fit(X_train, y_train)

y_pred = svr_gs.predict(X_test)

print(rmsle_metric(y_pred, y_test))

svr_gs.best_estimator_

KeyboardInterrupt: 